In [3]:

from benchmark import BenchmarkCase


bmcase = BenchmarkCase(
    'Prob030_popcount255', 
    'benchmarks/sample/Prob030_popcount255_spec.txt',
    'benchmarks/sample/Prob030_popcount255_ref.sv',
    'benchmarks/sample/Prob030_popcount255_tb.sv'
)

print(bmcase.specification)

I would like you to implement a module named TopModule with the following
interface. All input and output ports are one bit unless otherwise
specified.

 - input  in  (255 bits)
 - output out (  8 bits)

A "population count" circuit counts the number of '1's in an input
vector. The module should implement a population count circuit for a
255-bit input vector.


In [4]:

from pathlib import Path
from rechisel import initial_chisel_generation
from rechisel_chisel import ChiselCode


target_path = Path('output/gen.scala')

if target_path.exists():
    chisel_code = ChiselCode(f"```scala\n\n{target_path.read_text()}\n\n```", 'TopModule')
else:
    chisel_code = initial_chisel_generation(bmcase, 'TopModule')
    target_path.write_text(chisel_code.decorated)

print(chisel_code.decorated)


package TopModule

import chisel3._
import chisel3.util._

import chisel3.stage.ChiselStage

class TopModule extends RawModule {
  // Define the input and output ports
  val in = IO(Input(UInt(255.W))) // 255-bit input
  val out = IO(Output(UInt(8.W)))  // 8-bit output for population count

  // Implement the population count
  out := PopCount(in) // PopCount calculates the number of '1's in the input vector
}

object Main extends App {
    (new ChiselStage).emitVerilog(
      new TopModule,
      Array(
        "--target-dir", "generated",
        "--emission-options=disableMemRandomization,disableRegisterRandomization",
      )
    )
}



In [5]:

from rechisel_verifier import Verifier, VerifierWorkingSpace, VerifyResult


verifier = Verifier(
    VerifierWorkingSpace('output/chisel', 'output/iv'),
    'verilog-eval'
)

is_ok = (
    verifier.prepare(chisel_code, bmcase) and
    verifier.compile() and
    verifier.iv() and
    verifier.vvp() and
    verifier.functionality()
)
verify_result: VerifyResult = verifier.result

is_ok

True

In [6]:
import json

with open('output/verify.json', 'w') as fp:
    json.dump(verify_result.__dict__, fp, indent=2)